<a href="https://colab.research.google.com/github/aRandomAlexa/LEC-Hacks/blob/main/dogbreedmodel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
from google.colab import drive
drive.mount('/content/drive')
checkpoints = '/content/drive/MyDrive/colab_files/'
if not os.path.exists(checkpoints):
    os.makedirs(checkpoints)

In [ ]:
import os
import shutil
if not os.path.exists('imagenet64'):
  if not os.path.exists(checkpoints + 'images.tar'):
    print("Downloading archive...")
    os.chdir(checkpoints)
    !wget http://vision.stanford.edu/aditya86/ImageNetDogs/images.tar
    os.chdir('/content/')
  print("Copying to local runtime...")
  shutil.copy(checkpoints + 'images.tar', './images.tar')
  print("Uncompressing...")
  !tar -xf images.tar
print("Data ready!")

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Set hyperparameters
num_epochs = 45
batch_size = 256
learning_rate = 0.001

# Initialize transformations for data augmentation
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.RandomRotation(degrees=45),
    transforms.ColorJitter(brightness=0.5, contrast=0.5, saturation=0.5, hue=0.5),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Load the ImageNet Object Localization Challenge dataset
train_dataset = torchvision.datasets.ImageFolder(
    root='./Images',
    transform=transform
)


train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)

# Load the ResNet50 model
model = torchvision.models.resnet50(pretrained=True)

# Freeze the parameters of the model
for param in model.parameters():
    param.requires_grad = False

model.fc = torch.nn.Sequential(torch.nn.Linear(model.fc.in_features, 120))

# Set the model to run on the device
model = model.to(device)


model.load_state_dict(torch.load("/content/drive/MyDrive/colab_files/model.pt"));

# Define the loss function and optimizer
criterion = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

def train(network, data, loss_function, op, epochs):
    for epoch in range(epochs):
        counter = 0
        correct = 0
        total = 0
        # Loop over the dataloader
        for inputs, labels in data:

            #calculating loss/training
            network.train()
            inputs = inputs.to(device, non_blocking=True)
            labels = labels.to(device, non_blocking=True)
            # Forward pass
            outputs = model(inputs)
            loss = loss_function(outputs, labels)
            # Backward pass
            op.zero_grad()
            loss.backward()
            op.step()

            #getting accuracy
            network.eval()
            outputs = network(inputs)
            _, predicted = torch.max(outputs.data, 1)
            t = labels.size(0)
            c = (predicted == labels).sum().item()
            counter += 1

            total += t
            correct += c

            if (counter%10 == 0):
                print(f'Set [{epoch+1}, {counter}], Loss: {loss.item():.4f}, Acc: {c/t:.4f}')

        print(f'Average Acc: {correct/total:.4f}')
        torch.save(model.state_dict(), "/content/drive/MyDrive/colab_files/model.pt")


train(model, train_loader, criterion, optimizer, num_epochs);
def accuracy(net, dataloader):
    net.to(device)
    net.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        counter = 0;
        for batch in dataloader:
            images, labels = batch[0].to(device), batch[1].to(device)
            outputs = net(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            counter += 1;
            if (counter%10 == 0):
                print(f'Set [{counter}], Accuracy: {(predicted == labels).sum().item()/labels.size(0)}')
    return correct/total

torch.save(model.state_dict(), "/content/drive/MyDrive/colab_files/model.pt")

print(f'accuracy: {accuracy(model, train_loader)}')


In [ ]:
print(train_dataset.classes)

# prepare to count predictions for each class
correct_pred = {classname: 0 for classname in train_dataset.classes}
total_pred = {classname: 0 for classname in train_dataset.classes}

# again no gradients needed
with torch.no_grad():
    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.to(device)
        # calculate outputs by running images through the network
        outputs = model(images)
        _, predictions = torch.max(outputs, 1)
        # collect the correct predictions for each class
        for label, prediction in zip(labels, predictions):
            if label == prediction:
                correct_pred[train_dataset.classes[label]] += 1
            total_pred[train_dataset.classes[label]] += 1


# print accuracy for each class
for classname, correct_count in correct_pred.items():
    accuracy = 100 * float(correct_count) / total_pred[classname]
    print(f'Accuracy for class: {classname:5s} is {accuracy:.1f} %')

In [ ]:
from PIL import Image
import matplotlib
import numpy as np

img = Image.open("IMG_1784.jpeg")

img = transform(img)



if torch.cuda.is_available():
    model = model.cuda()
    img = img.cuda()  # Make sure the input data is also on the GPU
else:
    model = model.cpu()
    img = img.cpu()

img = img.unsqueeze(0)

model.eval()
outputs = model(img)
outputs = torch.nn.functional.softmax(outputs, dim=1)
_ , predicted = torch.max(outputs, 1)

print(f'Predicted: {train_dataset.classes[predicted.item()]}')


Predicted: n02098105-soft-coated_wheaten_terrier
